# 模型的加载和保存

In [ ]:
首先看detect.py的加载模型：
     if weights_file_path.endswith('.pt'):     
            if weights_file_path.endswith('weights/yolov3.pt') and not os.path.isfile(weights_file_path):           #但是键入的pt文件不存在
                os.system('wget https://storage.googleapis.com/ultralytics/yolov3.pt -O ' + weights_file_path)      #把pt权值文件下到指定文件夹下
            checkpoint = torch.load(weights_file_path, map_location='cpu')
            model.load_state_dict(checkpoint['model'])
            del checkpoint
        else:   # darknet format
            load_weights(model, weights_file_path)
    '''
    torch.load(weights_file_path, map_location='cpu')是加载整个模型到CPU上，返回一个模型结构和参数的字典
    查看一下checkpoint字典的keys： dict_keys(['epoch', 'best_loss', 'model', 'optimizer'])（epoch：当前跑了多少轮；best_loss：最优的loss）
    接下来model.load_state_dict(checkpoint['model'])，将存储的key为model的value数据信息加载到初始化好的model上
    (如果不是pt文件而是darknet的weights则需要解读加载，函数暂时没看)
    '''

# 优化器optimizer

In [ ]:
优化器创建：    
            optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr0, momentum=.9)
'''
看看optimizer输出是什么：SGD (Parameter Group 0
                            dampening: 0
                            lr: 0.001
                            momentum: 0.9
                            nesterov: False
                            weight_decay: 0）
分析：
1.先看filter（func,iter）：
    传入一个过滤函数和一个可迭代对象（如列表，字符串），将迭代对象的元素一个个代入函数作为输入，判断函数得到结果为True和False，只输出True的迭代对象
    在这里，判断函数是p.requires_grad输出bool，说明其是否需要求梯度，从而实现将model.parameters()中不用学训练的参数过滤掉
2.再看model.parameters()：
    parameters()方法通常用于将参数传入optimizer；
    返回一个包含模型所有参数的迭代器，一个generator对象，一般用来当作optimizer的参数。其结构是怎样的呢？我们遍历看一下：
    for param in model.parameters():
        print(type(param)，type(param.data),'\n\n', param.size(),'\n\n',param)
    得到结果：   
        param是一个torch.nn.parameter.Parameter对象
        <class 'torch.Tensor'>               #这个tensor就是下面的那个tensor了
        torch.Size([32, 3, 3, 3])            #第一层卷积是32个3*3卷积核，深度为3，
        Parameter containing:
            tensor([[[[nan, nan, nan],       #最后一项是requires_grad=True，说明第一个遍历的项需要求梯度
                      [nan, nan, nan],       
                      [nan, nan, nan]],
                        ...
                     [[nan, nan, nan],
                      [nan, nan, nan],
                      [nan, nan, nan]]]], device='cuda:0', requires_grad=True)
    我再遍历了下model.parameters()长度发现有222个元素，正好和层数对应，很明显了，也就是一个param存储一个层的卷积核参数，如下面cell
3.传递动量和学习率                                           
'''

优化器数据加载：
 if checkpoint['optimizer'] is not None:
            optimizer.load_state_dict(checkpoint['optimizer'])
            best_loss = checkpoint['best_loss']
            
'''
checkpoint字典包含四个key，其中optimizer的value打印出来发现也是一个dict，checkpoint['optimizer']的key为：
dict_keys(['state', 'param_groups'])，那么再分别看这两个：
    （1）checkpoint['optimizer']['state']
        type()查看类型，又是个dict，查看其key:[140148369314152, 140148369314224, ...，140146662313128],有222个，对应网络层数，取一个value看：
        key=140148369314152时，value又是一个字典，但是只有一个键值对：key=['momentum_buffer']，value是个张量，shape为：torch.Size([32, 3, 3, 3])
        也就是第一层卷积的32*3*3*3
    （2）checkpoint['optimizer']['param_groups']
        type()查看类型，是个list,但是list内元素是dict，内容如下：
            [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 
              'params': [140148369314152, 140148369314224, ...，140146662313128]}]
        第一行的都是优化器的参数，第二行params的value是长度222的列表，也是state的key

'''

# 训练网络结构打印

In [ ]:
'''
每个layer的shape是卷积核个数*w*h*channel，对于BN层等就只有channel了
'''
实现代码：
def model_info(model):  # Plots a line-by-line description of a PyTorch model
    n_p = sum(x.numel() for x in model.parameters())  # number parameters
    n_g = sum(x.numel() for x in model.parameters() if x.requires_grad)  # number gradients
    print('\n%5s %50s %9s %12s %20s %12s %12s' % ('layer', 'name', 'gradient', 'parameters', 'shape', 'mu', 'sigma'))
    for i, (name, p) in enumerate(model.named_parameters()):
        name = name.replace('module_list.', '')
        print('%5g %50s %9s %12g %20s %12.3g %12.3g' % (
            i, name, p.requires_grad, p.numel(), list(p.shape), p.mean(), p.std()))
    print('Model Summary: %g layers, %g parameters, %g gradients\n' % (i + 1, n_p, n_g))
    '''
    n_p ：sum()内的一串东西，是遍历每个层，然后得到该层矩阵的元素数目（如第一层为torch.Size([32, 3, 3, 3])， x=32*3*3*3）,
         对所有层进行求和得到总共需要设置的参数
    n_g ：类似上面，单只是计算x.requires_grad的个数，得到需要通过梯度下降学习的参数个数
    model.named_parameters()：(re)named_parameters() 是给出网络层的名字和参数的迭代器，相比而言上面的parameters()会给出一个网络的全部参数的选代器
          这个model.named_parameters()里面有两个东西，我们进行for遍历打印：
          （1）遍历打印全部的name:module_list.0.conv_0.weight,module_list.0.batch_norm_0.weight,...,module_list.105.conv_105.bias
              很明显，第一个参数name就是网络层的名字
          （2）打印打一个p：和上面parameters()方法的param.data一样了，得到的是torch.Size([32, 3, 3, 3])的张量存放卷积核等的参数
    '''
    
    

layer                                               name  gradient   parameters                shape           mu        sigma
    0                                    0.conv_0.weight      True          864        [32, 3, 3, 3]          nan          nan
    1                              0.batch_norm_0.weight      True           32                 [32]          nan          nan
    2                                0.batch_norm_0.bias      True           32                 [32]          nan          nan
    3                                    1.conv_1.weight      True        18432       [64, 32, 3, 3]          nan          nan
    4                              1.batch_norm_1.weight      True           64                 [64]    -1.47e+03     1.25e+04
    5                                1.batch_norm_1.bias      True           64                 [64]         -525     5.46e+03
    6                                    2.conv_2.weight      True         2048       [32, 64, 1, 1]          nan          nan
    7                              2.batch_norm_2.weight      True           32                 [32]         -792     9.58e+03
    8                                2.batch_norm_2.bias      True           32                 [32]         -454     4.43e+03
    9                                    3.conv_3.weight      True        18432       [64, 32, 3, 3]          nan          nan
   10                              3.batch_norm_3.weight      True           64                 [64]          nan          nan
   11                                3.batch_norm_3.bias      True           64                 [64]          nan          nan
   12                                    5.conv_5.weight      True        73728      [128, 64, 3, 3]          nan          nan
   13                              5.batch_norm_5.weight      True          128                [128]          nan          nan
   14                                5.batch_norm_5.bias      True          128                [128]          nan          nan
   15                                    6.conv_6.weight      True         8192      [64, 128, 1, 1]          nan          nan
   16                              6.batch_norm_6.weight      True           64                 [64]          nan          nan
   17                                6.batch_norm_6.bias      True           64                 [64]          nan          nan
   18                                    7.conht      True         8192      [64, 128, 1, 1]          nan          nan
   22                              9.batch_norm_9.weight      True           64                 [64]          nan          nan
   23                                9.batch_norm_9.bias      True           64                 [64]          nan          nan
   24                                  10.conv_10.weight      True        73728      [128, 64, 3, 3]          nan          nan
   25                            10.batch_norm_10.weight      True          128                [128]          nan          nan
   26                              10.batch_norm_10.bias      True          128                [128]          nan          nan
   27                                  12.conv_12.weight      True       294912     [256, 128, 3, 3]          nan          nan
   28                            12.batch_norm_12.weight      True          256                [256]          nan          nan
   29                              12.batch_norm_12.bias      True          256                [256]          nan          nan
   30                                  13.conv_13.weight      True        32768     [128, 256, 1, 1]          nan          nan
   31                            13.batch_norm_13.weight      True          128                [128]          nan          nan
   32                              13.batch_norm_13.bias      True          128                [128]          nan          nan
   33                                  14.conv_14.weight      True       294912     [256, 128, 3, 3]          nan          nan
   34                            14.batch_norm_14.weight      True          256                [256]          nan          nan
   35                              14.batch_norm_14.bias      True          256                [256]          nan          nan
   36                                  16.conv_16.weight      True        32768     [128, 256, 1, 1]          nan          nan
   37                            16.batch_norm_16.weight      True          128                [128]          nan          nan
   38                              16.batch_norm_16.bias      True          128                [128]          nan          nan
   39                                  17.conv_17.weight      True       294912     [256, 128, 3, 3]          nan          nan
   40                            17.batch_norm_17.weight      True          256                [256]          nan          nan
   41                              17.batch_norm_17.bias      True          256                [256]          nan          nan
   42                                  19.conv_19.weight      True        32768     [128, 256, 1, 1]          nan          nan
   43                            19.batch_norm_19.weight      True          128                [128]          nan          nan
   44                              19.batch_norm_19.bias      True          128                [128]          nan          nan
   45                                  20.conv_20.weight      True       294912     [256, 128, 3, 3]          nan          nan
   46                            20.batch_norm_20.weight      True          256                [256]          nan          nan
   47                              20.batch_norm_20.bias      True          256                [256]          nan          nan
   48                                  22.conv_22.weight      True        32768     [128, 256, 1, 1]          nan          nan
   49                            22.batch_norm_22.weight      True          128                [128]          nan          nan
   50                              22.batch_norm_22.bias      True          128                [128]          nan          nan
   51                                  23.conv_23.weight      True       294912     [256, 128, 3, 3]          nan          nan
   52                            23.batch_norm_23.weight      True          256                [256]          nan          nan
   53                              23.batch_norm_23.bias      True          256                [256]          nan          nan
   54                                  25.conv_25.weight      True        32768     [128, 256, 1, 1]          nan          nan
   55                            25.batch_norm_25.weight      True          128                [128]          nan          nan
   56                              25.batch_norm_25.bias      True          128                [128]          nan          nan
   57                                  26.conv_26.weight      True       294912     [256, 128, 3, 3]          nan          nan
   58                            26.batch_norm_26.weight      True          256                [256]          nan          nan
   59                              26.batch_norm_26.bias      True          256                [256]          nan          nan
   60                                  28.conv_28.weight      True        32768     [128, 256, 1, 1]          nan          nan
   61                            28.batch_norm_28.weight      True          128                [128]          nan          nan
   62                              28.batch_norm_28.bias      True          128                [128]          nan          nan
   63                                  29.conv_29.weight      True       294912     [256, 128, 3, 3]          nan          nan
   64                            29.batch_norm_29.weight      True          256                [256]          nan          nan
   65                              29.batch_norm_29.bias      True          256                [256]          nan          nan
   66                                  31.conv_31.weight      True        32768     [128, 256, 1, 1]          nan          nan
   67                            31.batch_norm_31.weight      True          128                [128]          nan          nan
   68                              31.batch_norm_31.bias      True          128                [128]          nan          nan
   69                                  32.conv_32.weight      True       294912     [256, 128, 3, 3]          nan          nan
   70                            32.batch_norm_32.weight      True          256                [256]          nan          nan
   71                              32.batch_norm_32.bias      True          256                [256]          nan          nan
   72                                  34.conv_34.weight      True        32768     [128, 256, 1, 1]          nan          nan
   73                            34.batch_norm_34.weight      True          128                [128]          nan          nan
   74                              34.batch_norm_34.bias      True          128                [128]          nan          nan
   75                                  35.conv_35.weight      True       294912     [256, 128, 3, 3]          nan          nan
   76                            35.batch_norm_35.weight      True          256                [256]          nan          nan
   77                              35.batch_norm_35.bias      True          256                [256]          nan          nan
   78                                  37.conv_37.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
   79                            37.batch_norm_37.weight      True          512                [512]          nan          nan
   80                              37.batch_norm_37.bias      True          512                [512]          nan          nan
   81                                  38.conv_38.weight      True       131072     [256, 512, 1, 1]          nan          nan
   82                            38.batch_norm_38.weight      True          256                [256]          nan          nan
   83                              38.batch_norm_38.bias      True          256                [256]          nan          nan
   84                                  39.conv_39.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
   85                            39.batch_norm_39.weight      True          512                [512]          nan          nan
   86                              39.batch_norm_39.bias      True          512                [512]          nan          nan
   87                                  41.conv_41.weight      True       131072     [256, 512, 1, 1]          nan          nan
   88                            41.batch_norm_41.weight      True          256                [256]          nan          nan
   89                              41.batch_norm_41.bias      True          256                [256]          nan          nan
   90                                  42.conv_42.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
   91                            42.batch_norm_42.weight      True          512                [512]          nan          nan
   92                              42.batch_norm_42.bias      True          512                [512]          nan          nan
   93                                  44.conv_44.weight      True       131072     [256, 512, 1, 1]          nan          nan
   94                            44.batch_norm_44.weight      True          256                [256]          nan          nan
   95                              44.batch_norm_44.bias      True          256                [256]          nan          nan
   96                                  45.conv_45.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
   97                            45.batch_norm_45.weight      True          512                [512]          nan          nan
   98                              45.batch_norm_45.bias      True          512                [512]          nan          nan
   99                                  47.conv_47.weight      True       131072     [256, 512, 1, 1]          nan          nan
  100                            47.batch_norm_47.weight      True          256                [256]          nan          nan
  101                              47.batch_norm_47.bias      True          256                [256]          nan          nan
  102                                  48.conv_48.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
  103                            48.batch_norm_48.weight      True          512                [512]          nan          nan
  104                              48.batch_norm_48.bias      True          512                [512]          nan          nan
  105                                  50.conv_50.weight      True       131072     [256, 512, 1, 1]          nan          nan
  106                            50.batch_norm_50.weight      True          256                [256]          nan          nan
  107                              50.batch_norm_50.bias      True          256                [256]          nan          nan
  108                                  51.conv_51.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
  109                            51.batch_norm_51.weight      True          512                [512]          nan          nan
  110                              51.batch_norm_51.bias      True          512                [512]          nan          nan
  111                                  53.conv_53.weight      True       131072     [256, 512, 1, 1]          nan          nan
  112                            53.batch_norm_53.weight      True          256                [256]          nan          nan
  113                              53.batch_norm_53.bias      True          256                [256]          nan          nan
  114                                  54.conv_54.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
  115                            54.batch_norm_54.weight      True          512                [512]          nan          nan
  116                              54.batch_norm_54.bias      True          512                [512]          nan          nan
  117                                  56.conv_56.weight      True       131072     [256, 512, 1, 1]          nan          nan
  118                            56.batch_norm_56.weight      True          256                [256]          nan          nan
  119                              56.batch_norm_56.bias      True          256                [256]          nan          nan
  120                                  57.conv_57.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
  121                            57.batch_norm_57.weight      True          512                [512]          nan          nan
  122                              57.batch_norm_57.bias      True          512                [512]          nan          nan
  123                                  59.conv_59.weight      True       131072     [256, 512, 1, 1]          nan          nan
  124                            59.batch_norm_59.weight      True          256                [256]          nan          nan
  125                              59.batch_norm_59.bias      True          256                [256]          nan          nan
  126                                  60.conv_60.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
  127                            60.batch_norm_60.weight      True          512                [512]          nan          nan
  128                              60.batch_norm_60.bias      True          512                [512]          nan          nan
  129                                  62.conv_62.weight      True  4.71859e+06    [1024, 512, 3, 3]          nan          nan
  130                            62.batch_norm_62.weight      True         1024               [1024]          nan          nan
  131                              62.batch_norm_62.bias      True         1024               [1024]          nan          nan
  132                                  63.conv_63.weight      True       524288    [512, 1024, 1, 1]          nan          nan
  133                            63.batch_norm_63.weight      True          512                [512]          nan          nan
  134                              63.batch_norm_63.bias      True          512                [512]          nan          nan
  135                                  64.conv_64.weight      True  4.71859e+06    [1024, 512, 3, 3]          nan          nan
  136                            64.batch_norm_64.weight      True         1024               [1024]          nan          nan
  137                              64.batch_norm_64.bias      True         1024               [1024]          nan          nan
  138                                  66.conv_66.weight      True       524288    [512, 1024, 1, 1]          nan          nan
  139                            66.batch_norm_66.weight      True          512                [512]          nan          nan
  140                              66.batch_norm_66.bias      True          512                [512]          nan          nan
  141                                  67.conv_67.weight      True  4.71859e+06    [1024, 512, 3, 3]          nan          nan
  142                            67.batch_norm_67.weight      True         1024               [1024]          nan          nan
  143                              67.batch_norm_67.bias      True         1024               [1024]          nan          nan
  144                                  69.conv_69.weight      True       524288    [512, 1024, 1, 1]          nan          nan
  145                            69.batch_norm_69.weight      True          512                [512]          nan          nan
  146                              69.batch_norm_69.bias      True          512                [512]          nan          nan
  147                                  70.conv_70.weight      True  4.71859e+06    [1024, 512, 3, 3]          nan          nan
  148                            70.batch_norm_70.weight      True         1024               [1024]          nan          nan
  149                              70.batch_norm_70.bias      True         1024               [1024]          nan          nan
  150                                  72.conv_72.weight      True       524288    [512, 1024, 1, 1]          nan          nan
  151                            72.batch_norm_72.weight      True          512                [512]          nan          nan
  152                              72.batch_norm_72.bias      True          512                [512]          nan          nan
  153                                  73.conv_73.weight      True  4.71859e+06    [1024, 512, 3, 3]          nan          nan
  154                            73.batch_norm_73.weight      True         1024               [1024]          nan          nan
  155                              73.batch_norm_73.bias      True         1024               [1024]          nan          nan
  156                                  75.conv_75.weight      True       524288    [512, 1024, 1, 1]          nan          nan
  157                            75.batch_norm_75.weight      True          512                [512]          nan          nan
  158                              75.batch_norm_75.bias      True          512                [512]          nan          nan
  159                                  76.conv_76.weight      True  4.71859e+06    [1024, 512, 3, 3]          nan          nan
  160                            76.batch_norm_76.weight      True         1024               [1024]          nan          nan
  161                              76.batch_norm_76.bias      True         1024               [1024]          nan          nan
  162                                  77.conv_77.weight      True       524288    [512, 1024, 1, 1]          nan          nan
  163                            77.batch_norm_77.weight      True          512                [512]          nan          nan
  164                              77.batch_norm_77.bias      True          512                [512]          nan          nan
  165                                  78.conv_78.weight      True  4.71859e+06    [1024, 512, 3, 3]          nan          nan
  166                            78.batch_norm_78.weight      True         1024               [1024]          nan          nan
  167                              78.batch_norm_78.bias      True         1024               [1024]          nan          nan
  168                                  79.conv_79.weight      True       524288    [512, 1024, 1, 1]          nan          nan
  169                            79.batch_norm_79.weight      True          512                [512]          nan          nan
  170                              79.batch_norm_79.bias      True          512                [512]          nan          nan
  171                                  80.conv_80.weight      True  4.71859e+06    [1024, 512, 3, 3]          nan          nan
  172                            80.batch_norm_80.weight      True         1024               [1024]          nan          nan
  173                              80.batch_norm_80.bias      True         1024               [1024]          nan          nan
  174                                  81.conv_81.weight      True        18432     [18, 1024, 1, 1]          nan          nan
  175                                    81.conv_81.bias      True           18                 [18]          nan          nan
  176                                  84.conv_84.weight      True       131072     [256, 512, 1, 1]          nan          nan
  177                            84.batch_norm_84.weight      True          256                [256]          nan          nan
  178                              84.batch_norm_84.bias      True          256                [256]          nan          nan
  179                                  87.conv_87.weight      True       196608     [256, 768, 1, 1]          nan          nan
  180                            87.batch_norm_87.weight      True          256                [256]          nan          nan
  181                              87.batch_norm_87.bias      True          256                [256]          nan          nan
  182                                  88.conv_88.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
  183                            88.batch_norm_88.weight      True          512                [512]          nan          nan
  184                              88.batch_norm_88.bias      True          512                [512]          nan          nan
  185                                  89.conv_89.weight      True       131072     [256, 512, 1, 1]          nan          nan
  186                            89.batch_norm_89.weight      True          256                [256]          nan          nan
  187                              89.batch_norm_89.bias      True          256                [256]          nan          nan
  188                                  90.conv_90.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
  189                            90.batch_norm_90.weight      True          512                [512]          nan          nan
  190                              90.batch_norm_90.bias      True          512                [512]          nan          nan
  191                                  91.conv_91.weight      True       131072     [256, 512, 1, 1]          nan          nan
  192                            91.batch_norm_91.weight      True          256                [256]          nan          nan
  193                              91.batch_norm_91.bias      True          256                [256]          nan          nan
  194                                  92.conv_92.weight      True  1.17965e+06     [512, 256, 3, 3]          nan          nan
  195                            92.batch_norm_92.weight      True          512                [512]          nan          nan
  196                              92.batch_norm_92.bias      True          512                [512]          nan          nan
  197                                  93.conv_93.weight      True         9216      [18, 512, 1, 1]          nan          nan
  198                                    93.conv_93.bias      True           18                 [18]          nan          nan
  199                                  96.conv_96.weight      True        32768     [128, 256, 1, 1]          nan          nan
  200                            96.batch_norm_96.weight      True          128                [128]          nan          nan
  201                              96.batch_norm_96.bias      True          128                [128]          nan          nan
  202                                  99.conv_99.weight      True        49152     [128, 384, 1, 1]          nan          nan
  203                            99.batch_norm_99.weight      True          128                [128]          nan          nan
  204                              99.batch_norm_99.bias      True          128                [128]          nan          nan
  205                                100.conv_100.weight      True       294912     [256, 128, 3, 3]          nan          nan
  206                          100.batch_norm_100.weight      True          256                [256]          nan          nan
  207                            100.batch_norm_100.bias      True          256                [256]          nan          nan
  208                                101.conv_101.weight      True        32768     [128, 256, 1, 1]          nan          nan
  209                          101.batch_norm_101.weight      True          128                [128]          nan          nan
  210                            101.batch_norm_101.bias      True          128                [128]          nan          nan
  211                                102.conv_102.weight      True       294912     [256, 128, 3, 3]          nan          nan
  212                          102.batch_norm_102.weight      True          256                [256]          nan          nan
  213                            102.batch_norm_102.bias      True          256                [256]          nan          nan
  214                                103.conv_103.weight      True        32768     [128, 256, 1, 1]          nan          nan
  215                          103.batch_norm_103.weight      True          128                [128]          nan          nan
  216                            103.batch_norm_103.bias      True          128                [128]          nan          nan
  217                                104.conv_104.weight      True       294912     [256, 128, 3, 3]          nan          nan
  218                          104.batch_norm_104.weight      True          256                [256]          nan          nan
  219                            104.batch_norm_104.bias      True          256                [256]          nan          nan
  220                                105.conv_105.weight      True         4608      [18, 256, 1, 1]          nan          nan
  221                                  105.conv_105.bias      True           18                 [18]          nan          nan
